In [0]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from datetime import datetime
from datetime import date
from datetime import time
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
travelPony = 'drive/My Drive/Colab Notebooks/Travel Pony Facebook.csv'
df = pd.read_csv(travelPony)
df.head()

,Start Date,End Date,Account,Campaign Name,Reach,Frequency,Impressions,Clicks,Unique Clicks,Amount Spent (USD),Page Likes,Page Engagement,Post Engagement,Post Likes,Post Comments,Post Shares,Photo Views,Website Clicks,Cost per Page Like (USD),Cost per Page Engagement (USD)
0,2013-12-02,2013-12-02,Bob Jackson,Cyber Weekend,278117,2.314303,643647,336,313,300.00,3,246,243,0,0,0,0,243,100.00,1.219512
1,2013-12-01,2013-12-01,Bob Jackson,Cyber Weekend,221255,2.079090,460009,376,348,300.00,6,215,209,0,0,0,0,209,50.00,1.395349
2,2013-11-30,2013-11-30,Bob Jackson,Cyber Weekend,267915,2.499069,669538,383,345,300.00,2,252,250,0,0,0,0,250,150.00,1.190476
3,2013-11-29,2013-11-29,Bob Jackson,Cyber Weekend,280332,2.365406,663099,301,275,300.00,3,257,254,0,0,0,0,254,100.00,1.167315
4,2013-11-22,2013-11-22,Bob Jackson,TravelPony: General Ads,14269,1.000420,14275,64,61,35.18,2,14,12,0,0,0,0,12,17.59,2.512857


In [0]:
df['CPI'] = df['Amount Spent (USD)']/df['Impressions']

In [0]:
df['Start Date'] = pd.to_datetime(df['Start Date'])
df['Weekday'] = df['Start Date'].dt.day_name()
#df.head()

In [0]:
groupedDf = df.groupby('Weekday', as_index=False).agg({'CPI':'sum'})
groupedDf

,Weekday,CPI
0,Friday,2.056639
1,Monday,1.713976
2,Saturday,1.311720
3,Sunday,1.755447
4,Thursday,1.950631
5,Tuesday,1.440979
6,Wednesday,1.761665


Friday is the most expensive day for CPI. Saturday is the least expensive day.



In [0]:
print( df[['Amount Spent (USD)', 'Reach', 'Frequency', 'Unique Clicks', 'Page Likes']].corr())

                    Amount Spent (USD)     Reach  ...  Unique Clicks  Page Likes
Amount Spent (USD)            1.000000  0.703124  ...       0.882993    0.757612
Reach                         0.703124  1.000000  ...       0.722249    0.304388
Frequency                     0.130201  0.334101  ...       0.135103    0.000182
Unique Clicks                 0.882993  0.722249  ...       1.000000    0.584614
Page Likes                    0.757612  0.304388  ...       0.584614    1.000000

[5 rows x 5 columns]


Unique Clicks has the highest correlation to Amount Spent. This means that unique clicks increase the amount spent on an ad campaign more than any other factor. This makes sense practically, because if you pay-per-click, then clicks should be the greatest determining factor in campaign spend. 

In [0]:
target = df['Unique Clicks']
indep = df[['Reach', 'Frequency']]

mod = sm.OLS(target, indep)
res = mod.fit()
print(res.summary())

                                 OLS Regression Results                                
Dep. Variable:          Unique Clicks   R-squared (uncentered):                   0.557
Model:                            OLS   Adj. R-squared (uncentered):              0.556
Method:                 Least Squares   F-statistic:                              2325.
Date:                Wed, 13 Nov 2019   Prob (F-statistic):                        0.00
Time:                        05:47:17   Log-Likelihood:                         -15973.
No. Observations:                3705   AIC:                                  3.195e+04
Df Residuals:                    3703   BIC:                                  3.196e+04
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

Frequency impacts Unique Clicks far more than reach, with a coefficient of ~3.61. Meaning, how often an ad is displayed strongly impacts the number of clicks. The effect of frequency far exceeds the effect of reach (i.e. how many people the ad was served to). 